## Importing libs & exploration

In [39]:
import json
import os
import random
import numpy as np
import pandas as pd
import gensim
import nltk

from gensim.models import CoherenceModel
from multiprocess import Pool
from tqdm import tqdm 
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from string import punctuation

In [231]:
SOCIAL_MEDIA = 'RBK'

with open(f'data_31.03/{SOCIAL_MEDIA}/result.json') as json_data:
    data = json.load(json_data)

In [232]:
data['messages'][10:15]

[{'id': 51676,
  'type': 'message',
  'date': '2022-02-24T05:11:00',
  'from': 'Медуза — LIVE',
  'from_id': 'channel1036240821',
  'text': [{'type': 'bold',
    'text': 'Путин продолжает перечислять претензии к США и НАТО\n'},
   '\nДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И\xa0(ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И\xa0(ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА\n\nВоенные операции в Ираке, Сирии, Ливии —а также снова расширение на восток, которого не должно было быть. \n\n«Обманули, а выражаясь народным языком, просто кинули. Часто можно слышать, что политика — грязное дело, но не до такой же степени». \n\nСША российский президент назвал «империей лжи»']},
 {'id': 51677,
  'type': 'message',
  'date': '2022-02-24T05:11:20',
  'edited': '2022-02-24T05:11:51',
  'from': 'Медуза — LIVE',
  'from_id': 'channel1036240821',
  'text': ['❗️❗️❗️',
   {'type': 'bold', 'text': 'В Киеве слышны взрывы, 

In [233]:
all_types = np.unique([t['type'] for m in data['messages'] for t in m['text'] if type(t) is dict])
all_types

array(['bold', 'email', 'hashtag', 'italic', 'link', 'mention', 'phone',
       'text_link', 'underline'], dtype='<U9')

In [234]:
all_email = np.unique([t['text'] for m in data['messages'] for t in m['text'] if type(t) is dict and t['type'] == 'email'])
all_email

array(['photodoc@meduza.io', 'rusnorm@thebell.io'], dtype='<U18')

In [277]:
from demoji import replace 


BAD_TEXT_TYPES = ['email', 'mention', 'strikethrough', 'hashtag', 'code', 'link']
UKR_CHARS = ['ї', 'є', 'і', 'ʼ', 'ґ']
FOREIGN_AGENT_MESSAGE_START = 'ДАННОЕ'
SUBSCRIBE_MESSAGE = 'УС > Подписаться'
UKRAINE24_BANNER_MESSAGE = 'Украина 24/7'


def clean_str(input_str):
    output_str = replace(input_str.strip().replace("\n", "."), '')
    
    if FOREIGN_AGENT_MESSAGE_START in output_str:
        return ''
    
    output_str = output_str.replace('\u200b', ' ') \
                           .replace('\xa0', ' ') \
                           .replace(SUBSCRIBE_MESSAGE, '') \
                           .replace(UKRAINE24_BANNER_MESSAGE, '') \
                           .strip() + ' '
    
    return output_str
    

def create_datasets_from_news_sources(news_sources_list):
    
    for news_source in tqdm(news_sources_list):
        extracted_data = {
            'date': [],
            'text': []
        }

        ukr_messages_inds = []
        
        with open(f'data_31.03/{news_source}/result.json') as json_data:
            data = json.load(json_data)
            
        for i, m in enumerate(data['messages']):
            texts = m['text']
            text = ''

            if isinstance(texts, str):
                text += clean_str(texts)
            else:
                for t in texts:
                    sub_text = ''

                    if type(t) is dict and t['type'] not in BAD_TEXT_TYPES and len(t['text']) > 0 and 'http' not in t['text']:
                        sub_text = t['text']

                    if type(t) is str and len(t) > 0:
                        sub_text = t

                    sub_text = clean_str(sub_text)

                    if len(sub_text) == 0:
                        continue

                    # if sub_text[-1] not in ['!', '?', '.', ',', ':', ';']:
                    #     sub_text += ' '

                    text += sub_text

            if text != '' and len(text) > 25:
                extracted_data['date'].append(m['date'])
                extracted_data['text'].append(text.strip())

            if any(c in UKR_CHARS for c in text):
                ukr_messages_inds.append(len(extracted_data['text']) - 1)

        extracted_data['date'] = np.delete(extracted_data['date'], ukr_messages_inds)
        extracted_data['text'] = np.delete(extracted_data['text'], ukr_messages_inds)

        df = pd.DataFrame(extracted_data)
        df['date'] = pd.to_datetime(df['date'])
        df.to_csv(f'./data_31.03/{news_source}/dataset.csv', index=False)

In [278]:
SOCIAL_MEDIA_SOURCES = [d for d in os.listdir('data_31.03') if d[0] != '.']
create_datasets_from_news_sources(SOCIAL_MEDIA_SOURCES)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [01:25<00:00,  7.79s/it]


In [267]:
extracted_data['date'] = np.delete(extracted_data['date'], ukr_messages_inds)
extracted_data['text'] = np.delete(extracted_data['text'], ukr_messages_inds)

In [268]:
df = pd.DataFrame(extracted_data)
df['date'] = pd.to_datetime(df['date'])
df.to_csv(f'./data_31.03/{SOCIAL_MEDIA}/dataset.csv', index=False)

In [269]:
df.text.to_list()

['Пентагон считает, что российские войска прибывают на территорию ДНР и ЛНР, но не выходят за ее пределы',
 '️Украина запросила срочное заседание Совбеза ООН в связи с обращением ДНР и ЛНР к РФ с просьбой оказать им военную помощь, сообщил глава МИД Украины Кулеба.',
 'Мобильная связь республиканского оператора "Феникс" перестала работать на территории Донецка, сообщил наш корреспондент. ..Дозвониться нельзя из центра и ряда других районов города. При этом с доступом в мобильный интернет проблем нет.',
 'Президент Украины Владимир Зеленский выступил с обращением',
 'Санкции ЕС за признание Россией ДНР и ЛНР вступили в силу по ссылке.',
 '️ЕС запретил своему бизнесу любые контакты с ДНР и ЛНР, включая торговлю, инвестиции, техническую помощь.',
 'Полная расшифровка обращения президента Украины Владимира Зеленского на русском языке «Если руководство России не хочет ради мира садиться за стол с нами, возможно, оно сядет за стол с вами».  «Кто от этого пострадает больше всего? Люди. Кто эт

In [207]:
!zip -r data_31.03.zip data_31.03 

updating: data_31.03/ (stored 0%)
updating: data_31.03/BBC/ (stored 0%)
updating: data_31.03/Medusa/ (stored 0%)
updating: data_31.03/NYT/ (stored 0%)
updating: data_31.03/NovayaGazeta/ (stored 0%)
updating: data_31.03/RBK/ (stored 0%)
updating: data_31.03/RIANews/ (stored 0%)
updating: data_31.03/ShockedUkraine/ (stored 0%)
updating: data_31.03/TACC/ (stored 0%)
updating: data_31.03/Ukraine24/ (stored 0%)
updating: data_31.03/UkraineNow/ (stored 0%)
updating: data_31.03/WashingtonPost/ (stored 0%)
updating: data_31.03/Ukraine24/result.json (deflated 84%)
updating: data_31.03/Ukraine24/lda_model/ (stored 0%)
updating: data_31.03/Ukraine24/lda_model/lda.model.id2word (deflated 48%)
updating: data_31.03/Ukraine24/lda_model/lda.model.state (deflated 18%)
updating: data_31.03/Ukraine24/lda_model/lda.model.expElogbeta.npy (deflated 14%)
updating: data_31.03/Ukraine24/lda_model/lda.model (deflated 53%)
updating: data_31.03/Ukraine24/.ipynb_checkpoints/ (stored 0%)
updating: data_31.03/Ukrain